# LIBRARIES

In [8]:
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import os
import nltk
import matplotlib.pyplot as plt
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch
from sklearn.model_selection import train_test_split
import torch.optim as optim
from transformers import BertTokenizer, BertModel

In [2]:
def load_text_data(file_path):
    try:
        return pd.read_csv(file_path, on_bad_lines='skip')
    except Exception as e:
        print(f"Failed to load CSV: {e}")
        return None

In [9]:
def load_bert_model(model_name="bert-base-uncased"):
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)
    return tokenizer, model

In [4]:
def load_bert_model(model_name="bert-base-uncased"):
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)
    return tokenizer, model

In [5]:
def get_bert_embeddings(texts, tokenizer, model, device='cpu'):
    model.to(device)
    model.eval()
    embeddings = []
    with torch.no_grad():
        for text in texts:
            # Ensure text is a string
            text = str(text) if pd.notnull(text) else ""
            inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
            outputs = model(**inputs)
            hidden_states = outputs.last_hidden_state
            mean_embedding = torch.mean(hidden_states, dim=1).squeeze().cpu().numpy()
            embeddings.append(mean_embedding)
    return np.array(embeddings)

In [6]:
def load_and_preprocess_image(image_path, size=(64, 64)):
    if not os.path.exists(image_path):
        return None
    try:
        with Image.open(image_path) as img:
            img = img.convert('RGB').resize(size)
            img_array = np.array(img, dtype=np.float32) / 255.0
            return img_array
    except IOError:
        print(f"Error opening or processing image {image_path}")
        return None

In [10]:
def dataset_generator(text_file_path, image_folder_path, tokenizer, model, batch_size=32):
    data = load_text_data(text_file_path)
    if data is None or data.empty:
        print("No data available or data loading failed.")
        return

    current_year = data['year'].max()
    popular_seasons = ['Summer', 'Winter']
    data['y'] = ((data['year'] == current_year) & data['season'].isin(popular_seasons)).astype(int)

    data['productDisplayName'] = data['productDisplayName'].astype(str)
    text_features = get_bert_embeddings(data['productDisplayName'].tolist(), tokenizer, model)

    batch_images = []
    batch_text_features = []
    batch_y = []
    indices = []

    for idx, row in data.iterrows():
        image_path = os.path.join(image_folder_path, f"{row['id']}.jpg")
        image_array = load_and_preprocess_image(image_path)
        if image_array is not None:
            batch_images.append(image_array)
            batch_text_features.append(text_features[idx])
            batch_y.append(row['y'])
            indices.append(idx)
            if len(batch_images) == batch_size:
                yield np.array(batch_text_features), np.array(batch_images), np.array(batch_y)
                batch_images, batch_text_features, batch_y = [], [], []

    if batch_images:
        yieldnp.array(batch_text_features), np.array(batch_images), np.array(batch_y)
tokenizer, bert_model = load_bert_model()

In [ ]:
dataset_path = r'C:\Users\John Justine\Downloads\archive (10)\styles.csv'
image_path = r'C:\Users\John Justine\Downloads\archive (10)\e-commerce\images'

gen = dataset_generator(dataset_path, image_path, tokenizer, bert_model, batch_size=32)
text_features, image_data_batch, y_batch = next(gen)

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, images):
        self.images = images

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        return self.images[idx]

def extract_features_and_display_images(images, model_name="resnet50", batch_size=32, device='cpu', num_images_to_display=6):
    model = getattr(models, model_name)(pretrained=True)
    model = nn.Sequential(*list(model.children())[:-1])
    model.to(device)
    model.eval()

    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    transformed_images = [transform(image) for image in images]
    dataset = CustomImageDataset(transformed_images)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    features = []
    plt.figure(figsize=(10, 5))
    with torch.no_grad():
        for batch_index, imgs in enumerate(loader):
            print(f"Processing batch {batch_index + 1}/{len(loader)}") 
            imgs = imgs.to(device)
            feats = model(imgs)
            feats = feats.view(feats.size(0), -1)
            features.append(feats.cpu().numpy())

            if batch_index == 0: 
                for i, image in enumerate(imgs.cpu()):
                    if i >= num_images_to_display:
                        break
                    plt.subplot(1, num_images_to_display, i + 1)
                    plt.imshow(transforms.ToPILImage()(image))
                    plt.axis('off')
        plt.show()

    return np.vstack(features)

In [ ]:
visual_features = extract_features_and_display_images(image_data_batch)


In [ ]:
class MultimodalFusionWithAttention(nn.Module):
    def __init__(self, text_feature_dim, visual_feature_dim, fused_dim):
        super().__init__()
        self.text_weight = nn.Parameter(torch.randn(text_feature_dim, fused_dim))
        self.visual_weight = nn.Parameter(torch.randn(visual_feature_dim, fused_dim))
        self.attention_weight = nn.Parameter(torch.randn(fused_dim, 1))
        self.activation = nn.Tanh()

    def forward(self, text_features, visual_features):
        text_transformed = torch.matmul(text_features, self.text_weight)
        visual_transformed = torch.matmul(visual_features, self.visual_weight)
        combined_features = self.activation(text_transformed + visual_transformed)
        attention_scores = torch.matmul(combined_features, self.attention_weight)
        attention_weights = F.softmax(attention_scores, dim=1) 
        fused_features = combined_features * attention_weights.expand_as(combined_features)
        return fused_features

In [ ]:
text_features = torch.rand(32, 768)  
visual_features = torch.rand(32, 2048)  

fusion_model = MultimodalFusionWithAttention(768, 2048, 1024)
fused_features = fusion_model(text_features.float(), visual_features.float())

print(fused_features.shape)


In [ ]:
class PurchaseIntentModel(nn.Module):
    def __init__(self):
    
        super(PurchaseIntentModel, self).__init__()
        self.fc1 = nn.Linear(1024, 256)  
        self.bn1 = nn.BatchNorm1d(256)   
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)   
        self.dropout = nn.Dropout(0.3)   
        self.fc3 = nn.Linear(128, 64)
        self.bn3 = nn.BatchNorm1d(64)    
        self.fc4 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.relu(self.bn1(self.fc1(x)))
        x = self.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = self.relu(self.bn3(self.fc3(x)))
        x = self.sigmoid(self.fc4(x))
        return x

model = PurchaseIntentModel()

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def train_model(model, criterion, optimizer, X_train, y_train, X_test, y_test, epochs=50):
    model.train()  
    train_losses = []
    val_losses = []

    for epoch in range(epochs):
        optimizer.zero_grad() 
        outputs = model(X_train)  
        loss = criterion(outputs.squeeze(), y_train)  
        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())

        model.eval() 
        with torch.no_grad():
            val_outputs = model(X_test)
            val_loss = criterion(val_outputs.squeeze(), y_test)
            val_losses.append(val_loss.item())
        
        model.train() 

        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch + 1}/{epochs}], Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')
    
    return train_losses, val_losses


In [ ]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

train_losses, val_losses = train_model(model, criterion, optimizer, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor)

In [ ]:
def predict(model, X_test):
    model.eval()  
    with torch.no_grad():  
        inputs = torch.tensor(X_test, dtype=torch.float32)
        outputs = model(inputs)
        predictions = outputs.squeeze().detach() 
    return predictions.numpy() 

predictions = predict(model, X_test)
predictions = torch.sigmoid(torch.from_numpy(predictions)) 
predicted_labels = (predictions > 0.5).long() 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import average_precision_score

predicted_labels = predicted_labels.numpy()
y_test_array = y_test 

accuracy = accuracy_score(y_test_array, predicted_labels)
precision = precision_score(y_test_array, predicted_labels)
recall = recall_score(y_test_array, predicted_labels)
f1 = f1_score(y_test_array, predicted_labels)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

average_precision = average_precision_score(y_test_array, predictions.numpy())

print(f"Mean Average Precision: {average_precision}")

from sklearn.metrics import ndcg_score
ndcg = ndcg_score(np.array([y_test_array]), np.array([predictions.numpy()]))

print(f"NDCG Score: {ndcg}")


In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Training and Validation Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Training and Validation Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.yscale('log')
plt.show()

In [ ]:
# Show the learned embeddings
embeddings = model.embeddings.weight.data.numpy()
target_names = np.unique(y_test_array)

plt.figure(figsize=(10, 10))
for i, target_name in enumerate(target_names):
    plt.scatter(embeddings[y_test_array == target_name, 0], 
                 embeddings[y_test_array == target_name, 1], 
                 label=target_name)

plt.legend()
plt.show()

def load_image(image_path, size=(224, 224)):
    if not os.path.exists(image_path):
        print(f"Error: The specified file does not exist - {image_path}")
        return None
    
    try:
        with Image.open(image_path) as img:
            img = img.convert('RGB').resize(size)
            img_array = np.array(img)
            img_array = img_array / 255.0

            return img_array
    except Exception as e:
        print(f"An error occurred while loading the image: {e}")
        return None

In [ ]:
def plot_multimodal_recommendations(images, descriptions, scores, top_n=5):
    fig = plt.figure(figsize=(15, 3 * top_n))
    gs = gridspec.GridSpec(top_n, 2, width_ratios=[1, 3])

    for i in range(top_n):
        if images[i] is not None: 
            ax_image = plt.subplot(gs[i, 0])
            ax_image.imshow(images[i])  
            ax_image.axis('off')

            ax_text = plt.subplot(gs[i, 1])
            ax_text.text(0.5, 0.5, f'{descriptions[i]}\nScore: {scores[i]:.2f}', 
                         verticalalignment='center', horizontalalignment='center',
                         fontsize=12, multialignment='left')
            ax_text.axis('off')
        else:
            print(f"Failed to load image at index {i}")
    
    plt.tight_layout()
    plt.show()

image_paths = [
    r'C:\Users\John Justine\Downloads\archive (10)\e-commerce\images\8759.jpg',
    r'C:\Users\John Justine\Downloads\archive (10)\e-commerce\images\8760.jpg',
    r'C:\Users\John Justine\Downloads\archive (10)\e-commerce\images\8761.jpg',
    r'C:\Users\John Justine\Downloads\archive (10)\e-commerce\images\8762.jpg',
    r'C:\Users\John Justine\Downloads\archive (10)\e-commerce\images\8763.jpg'
]

sample_images = [load_image(path) for path in image_paths]

sample_descriptions = [
    "Puma Men Grey T-shirt",
    "Jealous 21 Women Purple Shirt",
    "Rocky S Women White Handbag",
    "Gini and Jony Boy's Kaleb White Brown Kidswear",
    "Do U Speak Green Men Blue Shorts"
]
sample_scores = [0.95, 0.92, 0.88, 0.86, 0.85]
plot_multimodal_recommendations(sample_images, sample_descriptions, sample_scores, top_n=top_n)